In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep
import pandas as pd
import json
import numpy as np
import io
from Web.browse_edgar import *
import datetime

https://www.sec.gov/ixviewer/ix.html?doc= (/Archives/edgar/data/320193/000162828017000717/a10-qq1201712312016.htm) is prefix for xlbr viewer :)


# Trying sec website. not actual filings

https://www.macrotrends.net/stocks/charts/META/meta-platforms/financial-statements
go to all financial statements
interactive data :)

10-q - to co szukamy kwartalne
10-k - to co szukamy roczne

In [3]:
def scrape_income_statement(driver):
    # TODO: https://stackoverflow.com/questions/8474031/case-insensitive-xpath-contains-possible
    table = open_income_statement(driver)
    lines = []
    for tr in table.find_elements(By.XPATH, "./tr"):
        line = []
        for th in tr.find_elements(By.XPATH, "./th") or []:
            text = th.text.strip()
            if text:
                line.append(th.text)
        for td in tr.find_elements(By.XPATH, "./td") or []:
            text = td.text.strip()
            if text:
                line.append(td.text)
        lines.append(line)
        
    title = lines.pop(0)
    try:
        date = pd.to_datetime(lines[0][0], format='%b. %d, %Y')
        dates = lines.pop(0)
    except:
        dates = title[1:]
        print('No date in second line')
    
    try:
        lines = list(filter(lambda x: len(x) > 1, lines))
        
        lines = np.array(lines)
        lines = np.char.replace(lines, '$', '')
        np.char.strip(lines)
        df = pd.DataFrame(lines, columns=['name', *dates])
    except:
        print(lines)
        return None
    return df.set_index('name')
    # return df

In [4]:
def income_statement_to_df(driver):
    table = open_income_statement(driver)
    f = io.StringIO('<table>' + table.get_attribute('innerHTML') + '</table>')
    df = pd.read_html(f)
    return df[0]

In [5]:
def cleanup_df(df):
    # remove multiIndex...
    df.to_csv('df.csv')
    with open('df.csv', "r+") as f:
        d = f.readlines()
        f.seek(0)
        for i, line in enumerate(d):
            if i != 0:
                f.write(line)
        f.truncate()
    df = pd.read_csv('df.csv')
    df = df.drop(df.columns[0], axis=1)
    # not sure 
    df = df.dropna()
    df = df.set_index(df.columns[0])
    df.map(lambda x: float(x.replace('$', '').replace(',', '.')) if isinstance(x, str) and '$' in x else x)
    df = df.transpose()
    
    return df

In [6]:
def use_search_page(driver):
    buttons = driver.find_elements(By.XPATH, "//a[contains(text(), 'Interactive Data')]")
    dfs = []
    print(len(buttons))
    for i in range(len(buttons)):
        # try:
            new_buttons = driver.find_elements(By.XPATH, "//a[contains(text(), 'Interactive Data')]")
            new_buttons[i].click()
            sleep(2)
            open_sections(driver)
            df = income_statement_to_df(driver)
            df = cleanup_df(df)   
            if df is not None:
                dfs.append(df)
            # driver.back()
            driver.execute_script("window.history.go(-1)")
            driver.execute_script("window.history.go(-1)")
            print("done one")
            sleep(2)
        # except:
        #     continue
    return dfs

In [7]:
# inner join is enough - most important metric are in both
def parse_dfs(dfs):
    for df in dfs[1:]:
        cols_to_use = df.columns.difference(dfs[0].columns)
        dfs[0] = dfs[0].merge(df[cols_to_use], on='name')
    merged_t = dfs[0].transpose()
    
    merged_t = merged_t.assign(date=pd.to_datetime(merged_t.index))
    merged_t.set_index('date', inplace=True)
    return merged_t

In [8]:
lastDf = None

In [15]:
# TODO: ':' ten field jest przedrostkiem, grupa
# To nie osobna wartosc tylko: earnings per share -> Basic / Diluted. Chociaż to jest Basic (in dollars per share) XD

Index(['Dec. 31, 2023', 'Dec. 31, 2022', 'Dec. 31, 2021'], dtype='object')

In [12]:
driver = webdriver.Chrome()
search_url = 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=$$$&type=10-K&dateb=&owner=exclude&count=100'
# tickers = ['AMZN', 'META']
tickers = ["META"] # working
companies = {}
for ticker in tickers:
    url = search_url.replace('$$$', ticker)
    print(url)
    driver.get(url)
    dfs = use_search_page(driver)
    lastDf = dfs
    # companies[ticker] = parse_dfs(dfs)
dfs
# print("len", len(dfs))

https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=META&type=10-K&dateb=&owner=exclude&count=100
12
done clicking


CONSOLIDATED STATEMENTS OF INCOME - USD ($)  shares in Millions, $ in Millions  \
   CONSOLIDATED STATEMENTS OF INCOME - USD ($)  shares in Millions, $ in Millions   
0                         Income Statement [Abstract]                               
1                                             Revenue                               
2                                 Costs and expenses:                               
3                                     Cost of revenue                               
4                            Research and development                               
5                                 Marketing and sales                               
6                          General and administrative                               
7                            Total costs and expenses                               
8                              Income from operations                               
9            Interest and other income (expense), net                               
10           Income before provision for income taxes                               
11                         Provision for income taxes                               
12                                         Net income                               
13  Earnings per share attributable to Class A and...                               
14                       Basic (in dollars per share)                               
15                     Diluted (in dollars per share)                               
16  Weighted-average shares used to compute earnin...                               
17                                  Basic (in shares)                               
18                                Diluted (in shares)                               

   12 Months Ended                              
     Dec. 31, 2023 Dec. 31, 2022 Dec. 31, 2021  
0              NaN           NaN           NaN  
1        $ 134,902     $ 116,609     $ 117,929  
2              NaN           NaN           NaN  
3            25959         25249         22649  
4            38483         35338         24655  
5            12301         15262         14043  
6            11408         11816          9829  
7            88151         87665         71176  
8            46751         28944         46753  
9              677         (125)           531  
10           47428         28819         47284  
11            8330          5619          7914  
12        $ 39,098      $ 23,200      $ 39,370  
13             NaN           NaN           NaN  
14         $ 15.19        $ 8.63       $ 13.99  
15         $ 14.87        $ 8.59       $ 13.77  
16             NaN           NaN           NaN  
17            2574          2687          2815  
18            2629          2702          2859

In [ ]:
companies['META'].head()